The `pidgin.environment` establishes how `jinja2` injects data into templates for weaving.

templating happens before weaving.

`pidgin.rendering` controls the execution of `jinja2` and `mistune`.

The template needs to be created with from_string using globals and a special template_clas

In [3]:
    if __name__ == '__main__':
        %reload_ext pidgin
        import pidgin, IPython
        ip = get_ipython()
        ip.weave.html = True

In [ ]:
## Templating

Putting real data into documents

Templates are common in blogs.

Jinja has a port in javascript nunjucks.


[templating]: https://en.wikipedia.org/wiki/Template_processor
[jekyll]: https://jekyllrb.com/
[fstring]:

In [3]:
    import jinja2, nbconvert, types, collections, IPython
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]

In [ ]:
    def load_ipython_extension(ip): 
        mimebundle_formatter = ip.display_formatter.mimebundle_formatter
        for object in formatters_by_name: mimebundle_formatter.for_type_by_name(*object)  
        
    def unload_ipython_extension(ip=None): 
        mimebundle_formatter = ip.display_formatter.mimebundle_formatter
        for object in formatters_by_name: mimebundle_formatter.for_type_by_name(*object)

In [ ]:
    class TemplateExporter(nbconvert.exporters.TemplateExporter):
        def _create_environment(self):
            environment = super()._create_environment()
            environment.undefined = jinja2.StrictUndefined
            environment.finalize = ipython_output
            return environment
        
        @classmethod
        def new_environment(cls, **kwargs): return cls(**kwargs).environment

    new_environment = TemplateExporter.new_environment

In [ ]:
    def flatten(str): return '&#xA;'.join(str.splitlines())

In [ ]:
    def format_bytes(bytes, object):
        return {
            'text/html': format_images('image/png', {'image/png': bytes}),
            'text/plain': repr(object),}, {}

In [ ]:
    def show_axes(object):
        ip = IPython.get_ipython()
        import matplotlib.backends.backend_agg
        bytes = io.BytesIO()
        matplotlib.backends.backend_agg.FigureCanvasAgg(getattr(object, 'figure', object)).print_png(bytes)
        return format_bytes(bytes.getvalue(), object)

In [ ]:
    def show_sympy_axes(object): 
        s = __import__('io').BytesIO()
        object.save(s)
        return format_bytes(s.getvalue(), object)

In [ ]:
    formatters_by_name = (
        ('matplotlib.figure', 'Axes', show_axes), 
        ('matplotlib.axes._subplots', 'AxesSubplot', show_axes),
        ('sympy.plotting.plot', 'Plot', show_sympy_axes),
    )

In [ ]:
    if __name__ == '__main__':
        with tangle.PidginImporter():
            from . import weave
        unload_ipython_extension(get_ipython())        
        weave.load_ipython_extension(get_ipython())
        load_ipython_extension(get_ipython())

In [4]:
    def ipython_output(object=None, bundle=None):
        if bundle is None:             
            bundle, metadata = IPython.get_ipython().display_formatter.format(object)
        for type in [
            str for str in IPython.get_ipython().display_formatter.active_types if str != 'text/plain'
        ] + ['text/plain']:
            if type == 'text/plain': 
                if object is None: return bundle[type]
                return object
            if type in bundle: 
                object = bundle[type]
                if type.startswith('image') and 'svg' not in type: object = format_images(type, bundle)._repr_html_()
                if type ==  'text/html': object = flatten(object)
                if type == 'image/svg+xml': object = flatten(re.sub('(xmlns(:?\S*)?="\S*?")?', r'', object))
                if type == 'text/latex':
                    if object.startswith('$$') and object.endswith('$$'): object = object[1:-1]
                break
        return object

In [ ]:
    def format_images(type, bundle):
        str = bundle[type]
        import base64, IPython
        
        if isinstance(str, bytes):
            str = base64.b64encode(str).decode('utf-8')
            
        if type in ('image/svg+xml', 'text/html'):  ...
        elif str.startswith('http'): 
            str = F"""<img src="{str}"/>"""
        else: 
            str = F"""<img src="data:{type};base64,{str}"/>"""
            
        return IPython.display.HTML(str)